In [30]:
### import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
from glob import glob

#np.random.seed(5)

In [31]:
train_dir = os.path.join('./data_origin/train/')
test_dir = os.path.join('./data_origin/test')

#train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
train_datagen = ImageDataGenerator(
      rescale=1./255)      # 영상의 전체 픽셀을 255로 나눈다 - 영상이 0~1사이의 범위를 가지게 된다(normalization)
#      rotation_range=40,
 #     width_shift_range=0.2,
  #    height_shift_range=0.2,
   #   shear_range=0.2,
    #  zoom_range=0.2,
     # horizontal_flip=True,
      #fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(90,72),
                                                   batch_size=100,
                                                   class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(90,72),
                                                 batch_size=10,
                                                 class_mode='categorical')


Found 7200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [32]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (100, 90, 72, 3)
배치 레이블 크기: (100, 8)


In [33]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=(90,72,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 88, 70, 32)        896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 86, 68, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 43, 34, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 93568)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               11976832  
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 1032      
Total params: 11,997,256
Trainable params: 11,997,256
Non-trainable params: 0
________________________________________________________________

In [35]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [36]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch=20,
                           epochs=10,
                           validation_data=test_generator,
                           validation_steps=100)

Epoch 1/10
20/20 [==============================] - 47s 2s/step - loss: 1.8318 - acc: 0.4080 - val_loss: 0.9248 - val_acc: 0.5360
Epoch 2/10
20/20 [==============================] - 33s 2s/step - loss: 0.9175 - acc: 0.5800 - val_loss: 0.7930 - val_acc: 0.6510
Epoch 3/10
20/20 [==============================] - 32s 2s/step - loss: 0.7794 - acc: 0.6520 - val_loss: 0.6810 - val_acc: 0.6610
Epoch 4/10
20/20 [==============================] - 33s 2s/step - loss: 0.6723 - acc: 0.7065 - val_loss: 0.6095 - val_acc: 0.7180
Epoch 5/10
20/20 [==============================] - 32s 2s/step - loss: 0.6293 - acc: 0.7230 - val_loss: 0.5932 - val_acc: 0.7200
Epoch 6/10
20/20 [==============================] - 33s 2s/step - loss: 0.5907 - acc: 0.7430 - val_loss: 0.6645 - val_acc: 0.7090
Epoch 7/10
20/20 [==============================] - 32s 2s/step - loss: 0.5999 - acc: 0.7255 - val_loss: 0.5977 - val_acc: 0.7210
Epoch 8/10
20/20 [==============================] - 32s 2s/step - loss: 0.5326 - acc: 0.76

In [37]:
scores = model.evaluate_generator(test_generator, steps=10)
print(scores)

[0.4281902343034744, 0.8199999988079071]
